In [1]:
# Установить необходимые модули
# !pip install pyspark
# !pip install sqlalchemy
# !pip install psycopg2-binary

In [2]:
from sqlalchemy import create_engine, select
from sqlalchemy.orm import sessionmaker # функция, создающая новый экземпляр Session

from sqlalchemy import create_engine,Column,Integer,String,DateTime,Float,ForeignKey # ORM 
from sqlalchemy.orm import DeclarativeBase # базовый класс для создания декларативных моделей SQLAlchemy


import pandas as pd
from pathlib import Path

from sqlalchemy import func

In [3]:
# Параметры подключения к БД
pg_user = 'mluser'
pg_password = 'mlpassword'
pg_host = 'localhost'
pg_database = 'homework_dbms'
pg_port = '5433'
pg_connection_string = f'postgresql+psycopg2://{pg_user}:{pg_password}@{pg_host}:{pg_port}/{pg_database}'


In [4]:
# Подключение к БД и создание ORM-сессии
engine = create_engine(pg_connection_string, echo=False)

'''
sessionmaker() создает функцию для создания сессий ORM.
autocommit=False отключает автоматическое подтверждение транзакций.
autoflush=False отключает автоматическое обновление состояния сессии после каждого запроса.
bind=engine связывает сессию с созданным движком.
'''
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

In [5]:
class BaseTable(DeclarativeBase):
    pass

class Customer(BaseTable):
    __tablename__ = 'customers' # имя таблицы

    customer_id = Column(Integer, primary_key=True)
    first_name = Column(String(80))
    last_name = Column(String(80))
    phone = Column(String(80), nullable=True)
    email = Column(String(120), unique=True)
    street = Column(String(80))
    city = Column(String(80))
    state = Column(String(10))
    city = Column(String(80))
    zip_code = Column(Integer)

    def __repr__(self):
        '''
        Выводы класса в шаблон
        '''
        return f"<User(id={self.id}, email='{self.email}')>"
    
class Order(BaseTable):
    __tablename__ = 'orders' # имя таблицы

    order_id = Column(Integer, primary_key=True)
    customer_id = Column(Integer, ForeignKey('customers.customer_id'), nullable=True)
    order_status = Column(Integer)
    order_date = Column(DateTime)
    required_date = Column(DateTime)
    shipped_date = Column(DateTime, nullable=True)
    store_id = Column(Integer, ForeignKey('stores.store_id'))
    staff_id = Column(Integer, ForeignKey('staffs.staff_id')) 

    def __repr__(self):
        '''
        Выводы класса в шаблон
        '''
        return f"<Order(id={self.order_id}')>"
    
class Staff(BaseTable):
    __tablename__ = 'staffs' # имя таблицы

    staff_id = Column(Integer, primary_key=True)
    first_name = Column(String(80))
    last_name = Column(String(80))
    email = Column(String(120), unique=True)
    phone = Column(String(80))
    active = Column(Integer)
    store_id = Column(Integer, ForeignKey('stores.store_id'))
    manager_id = Column(Integer, ForeignKey('staffs.staff_id'), nullable=True)

    def __repr__(self):
        '''
        Выводы класса в шаблон
        '''
        return f"<Staf(id={self.staff_id}, email={self.email}')>" 
    
class Store(BaseTable):
    __tablename__ = 'stores' # имя таблицы

    store_id = Column(Integer, primary_key=True)
    store_name = Column(String(80))
    phone = Column(String(80))
    email = Column(String(120), unique=True)
    street = Column(String(80))
    city = Column(String(80))
    state = Column(String(10))
    zip_code = Column(Integer)

    def __repr__(self):
        '''
        Выводы класса в шаблон
        '''
        return f"<Store(id={self.id}, name={self.store_name}')>" 
    
class OrderItem(BaseTable):
    __tablename__ = 'order_items' # имя таблицы

    order_id = Column(Integer, ForeignKey('orders.order_id'), primary_key=True)
    item_id = Column(Integer, primary_key=True)
    product_id = Column(Integer, ForeignKey('products.product_id'))
    quantity = Column(Integer)
    list_price = Column(Float)
    discount = Column(Float)

    def __repr__(self):
        '''
        Выводы класса в шаблон
        '''
        return f"<OrderItem(order_id={self.oredr_id}, item_id={self.item_id}')>" 
    
class Category(BaseTable):
    __tablename__ = 'categories' # имя таблицы

    category_id = Column(Integer, primary_key=True, nullable=False)
    category_name = Column(String(80))

    def __repr__(self):
        '''
        Выводы класса в шаблон
        '''
        return f"<Category(id={self.id}, name='{self.name}')>"
    
class Product(BaseTable):
    __tablename__ = 'products' # имя таблицы

    product_id = Column(Integer, primary_key=True)
    product_name = Column(String(80))
    brand_id = Column(Integer, ForeignKey('brands.brand_id'))
    category_id = Column(Integer, ForeignKey('categories.category_id'), nullable=False)
    model_year = Column(Integer)
    list_price = Column(Float)

    def __repr__(self):
        '''
        Выводы класса в шаблон
        '''
        return f"<Product(id={self.product_id}, name='{self.product_name}')>"
    
class Stock(BaseTable):
    __tablename__ = 'stocks' # имя таблицы

    store_id = Column(Integer, ForeignKey('stores.store_id'), primary_key=True)
    product_id = Column(Integer, ForeignKey('products.product_id'), primary_key=True)
    quantity = Column(Integer)

    def __repr__(self):
        '''
        Выводы класса в шаблон
        '''
        return f"<Stock(store_id={self.store_id}, product_id='{self.product_id}')>"
    
class Brand(BaseTable):
    __tablename__ = 'brands' # имя таблицы

    brand_id = Column(Integer, primary_key=True)
    brand_name = Column(String(80), unique=True)

    def __repr__(self):
        '''
        Выводы класса в шаблон
        '''
        return f"<Brand(id={self.brand_id}, name='{self.brand_name}')>"

In [6]:
# Создание таблиц
BaseTable.metadata.drop_all(engine)
BaseTable.metadata.create_all(engine)


In [7]:
# Наполнение таблиц
data_directory = 'data'
tables = ['brands', 'categories', 'stores', 'products', 'customers', 'staffs', 'orders', 'order_items', 'stocks']
for table in tables:
    print(f'Загрузка данных в таблицу {table}')
    df = pd.read_csv(Path(data_directory, f'{table}.csv'))
    df.to_sql(table, engine, index=False, if_exists='append')

Загрузка данных в таблицу brands
Загрузка данных в таблицу categories
Загрузка данных в таблицу stores
Загрузка данных в таблицу products
Загрузка данных в таблицу customers
Загрузка данных в таблицу staffs
Загрузка данных в таблицу orders
Загрузка данных в таблицу order_items
Загрузка данных в таблицу stocks
